<a href="https://colab.research.google.com/github/cagBRT/PySpark/blob/master/Distrib_vs_Single.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we compare doing a task in a distributed environment and on a single CPU (single computer)

In [1]:
!git clone https://github.com/cagBRT/PySpark.git

Cloning into 'PySpark'...
remote: Enumerating objects: 426, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 426 (delta 60), reused 0 (delta 0), pack-reused 330
Receiving objects: 100% (426/426), 3.38 MiB | 15.86 MiB/s, done.
Resolving deltas: 100% (258/258), done.




---



---



# Setup PySpark<br>

If using DataBricks this section will be different

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a8f2d1c8f464d93780100a72abfee1f84de4977dae014d17a2c8f878c695b0a1
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
#Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
#getOrCreate gets or creates a session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [9]:
import urllib.request
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [10]:
spark = (
    SparkSession.builder
        .appName('learn')
        .config('spark.driver.memory', '8g')
        .master('local[4]',)
        .config('spark.sql.execution.arrow.pyspark.enabled', True)
        .config('spark.sql.execution.arrow.pyspark.fallback.enabled', False)
        .getOrCreate()
)

In [11]:
sc=spark.sparkContext.getOrCreate()

In [12]:
import time



---



---



We will now get all the works of Shakespeare (a very large file) and perform count and sort tasks.

# Get all of Shakespeare’s works

**Let's find out how long it takes a distributed computer to perform the task of counting all the words in Shakespeare's works**.

In [28]:
start_time = time.time()
#Count the number of words in all of Shakespear's works
Words=sc.textFile("/content/PySpark/shakespeare.txt")
WordsCount=Words.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
print("Number of words:",WordsCount.count())

--- 0.04732036590576172 seconds ---
Number of words: 1418390


**Let's find out how long it takes a *single computer* to perform the task of counting all the words in Shakespeare's works**.

In [29]:
start_time = time.time()
f= open("/content/PySpark/shakespeare.txt", "r")
words_shakes= f.read()
f.close()
word_shakes_python=words_shakes.split(" ")
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(word_shakes_python)

--- 0.0781714916229248 seconds ---


1293935

In [30]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.047492265701293945
single compute time:  0.0783390998840332


In [33]:
start_time = time.time()
#Count the number of distinct words
DistinctWordsCount=WordsCount.reduceByKey(lambda a,b: a+b)
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
print("Number of distinct words:",DistinctWordsCount.count())

--- 0.021169424057006836 seconds ---
Number of distinct words: 67506


In [34]:
start_time = time.time()
words_unique=set(word_shakes_python)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(words_unique)

--- 0.06757402420043945 seconds ---


86196

In [35]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.02133345603942871
single compute time:  0.06775259971618652


In [36]:
start_time = time.time()
#Sort the words by most-to-least words
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
#print most frequent 20 words
SortedWordsCount.top(20)

--- 0.9317646026611328 seconds ---


[(517065, ''),
 (23242, 'the'),
 (19540, 'I'),
 (18297, 'and'),
 (15623, 'to'),
 (15544, 'of'),
 (12532, 'a'),
 (10824, 'my'),
 (9576, 'in'),
 (9081, 'you'),
 (7851, 'is'),
 (7531, 'that'),
 (7068, 'And'),
 (6948, 'not'),
 (6722, 'with'),
 (6218, 'his'),
 (6009, 'your'),
 (6002, 'be'),
 (5616, 'for'),
 (5236, 'have')]

In [37]:
#start_time = time.time()
#counted_words=[]
#words_unique=list(words_unique)
#for i in range(len(words_unique)):
#  tuples= (word_shakes_python.count(words_unique[i]), words_unique[i])
#  counted_words.append(tuples)
#print("--- %s seconds ---" % (time.time() - start_time))
#sc_time= time.time() - start_time

In [38]:
#print("distributed time:    ",dc_time )
#print("single compute time: ", sc_time)

distributed time:     1.38<br>
single compute time:  2158



---

# Gettyburg Address

---



**Count all the words using a distributed environment**

In [15]:
start_time = time.time()
Lincoln = sc.textFile("/content/PySpark/GettysBurg.txt")
LincolnCount=Lincoln.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
dist_lincoln_count=LincolnCount.count()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
print("Number of words:",dist_lincoln_count)

--- 0.7262077331542969 seconds ---
Number of words: 266


**Count all the words using a single CPU**

In [16]:
start_time = time.time()
f= open("/content/PySpark/GettysBurg.txt", "r")
single_lincoln_count= f.read()
f.close()
single_lincoln_count=single_lincoln_count.split(" ")
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(single_lincoln_count)

--- 0.0005002021789550781 seconds ---


263

In [17]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.7264742851257324
single compute time:  0.003662586212158203


**Count unique words using a distributed environment**

In [18]:
#Count the number of distinct words
start_time = time.time()
DistinctWordsCount=LincolnCount.reduceByKey(lambda a,b: a+b)
DistinctWordsCount.count()
dc_time=time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))
DistinctWordsCount.count()

--- 1.836832046508789 seconds ---


158

**Count the unique words using a single computer**

In [19]:
start_time = time.time()
words_unique=set(single_lincoln_count)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(words_unique)

--- 0.0002200603485107422 seconds ---


155

In [20]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     1.8367347717285156
single compute time:  0.0017154216766357422


**Count the word frequency using a distributed environment**

In [21]:
start_time = time.time()
#Sort the words by most-to-least words
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
#print most frequent 20 words
#SortedWordsCount.top(20)

--- 1.283869981765747 seconds ---


**Count the word frequency using a single computer**

In [22]:
start_time = time.time()
counted_words=[]
words_unique=list(words_unique)
for i in range(len(words_unique)):
  tuples= (words_unique.count(words_unique[i]), words_unique[i])
  counted_words.append(tuples)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time= time.time() - start_time


--- 0.0012149810791015625 seconds ---


In [23]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     1.284203290939331
single compute time:  0.0024557113647460938
